# Importing Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Standardizing and Encoding Dataset

In [2]:
df_cleaned = pd.read_excel('../data/dataset_v1.xlsx', index_col = 'Loan_ID')
df_cleaned.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,146.369492,360,1,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.000000,360,1,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.000000,360,1,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.000000,360,1,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.000000,360,1,Urban,Y


In [3]:
x = df_cleaned.drop('Loan_Status', axis = 1)
y = df_cleaned['Loan_Status'].copy()

In [4]:
num_values = x.select_dtypes(include = np.number)
cat_values = x.select_dtypes(exclude = np.number)

In [5]:
cat_values.shape, num_values.shape, x.shape

((611, 6), (611, 5), (611, 11))

## Handling The Categorical Columns

In [6]:
cat_values.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area
Loan_ID,,,,,,
LP001002,Male,No,0,Graduate,No,Urban
LP001003,Male,Yes,1,Graduate,No,Rural
LP001005,Male,Yes,0,Graduate,Yes,Urban
LP001006,Male,Yes,0,Not Graduate,No,Urban
LP001008,Male,No,0,Graduate,No,Urban


In [7]:
def convert_to_0_1(row):
    return 1 if (row == 'Male') or (row == 'Yes') else 0

In [8]:
cat_values["Married"] = cat_values["Married"].apply(convert_to_0_1)
cat_values["Self_Employed"] = cat_values["Self_Employed"].apply(convert_to_0_1)
cat_values["Gender"] = cat_values["Gender"].apply(convert_to_0_1)


In [9]:
cat_values.head()

,Gender,Married,Dependents,Education,Self_Employed,Property_Area
Loan_ID,,,,,,
LP001002,1,0,0,Graduate,0,Urban
LP001003,1,1,1,Graduate,0,Rural
LP001005,1,1,0,Graduate,1,Urban
LP001006,1,1,0,Not Graduate,0,Urban
LP001008,1,0,0,Graduate,0,Urban


In [10]:
cat_values_ordinal = cat_values.loc[:, ["Gender", "Married", "Self_Employed"]]
cat_values_to_one_hot = cat_values.drop(["Gender", "Married", "Self_Employed"], axis = 1)

In [11]:
cat_values_to_one_hot = pd.get_dummies(cat_values_to_one_hot)

In [12]:
cat_values_to_one_hot.head()

,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,
LP001002,True,False,False,False,True,False,False,False,True
LP001003,False,True,False,False,True,False,True,False,False
LP001005,True,False,False,False,True,False,False,False,True
LP001006,True,False,False,False,False,True,False,False,True
LP001008,True,False,False,False,True,False,False,False,True


In [13]:
cat_values_ordinal.head()

,Gender,Married,Self_Employed
Loan_ID,,,
LP001002,1,0,0
LP001003,1,1,0
LP001005,1,1,1
LP001006,1,1,0
LP001008,1,0,0


In [14]:
perfect_cat_values = pd.concat([cat_values_ordinal, cat_values_to_one_hot], axis = 1)

In [15]:
perfect_cat_values.head()

,Gender,Married,Self_Employed,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,
LP001002,1,0,0,True,False,False,False,True,False,False,False,True
LP001003,1,1,0,False,True,False,False,True,False,True,False,False
LP001005,1,1,1,True,False,False,False,True,False,False,False,True
LP001006,1,1,0,True,False,False,False,False,True,False,False,True
LP001008,1,0,0,True,False,False,False,True,False,False,False,True


# Handling The Numerical Columns

In [16]:
num_values.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
Loan_ID,,,,,
LP001002,5849,0.0,146.369492,360,1
LP001003,4583,1508.0,128.000000,360,1
LP001005,3000,0.0,66.000000,360,1
LP001006,2583,2358.0,120.000000,360,1
LP001008,6000,0.0,141.000000,360,1


In [17]:
num_values.Credit_History.value_counts()

Credit_History
1    494
0    117
Name: count, dtype: int64

In [18]:
df_cleaned.shape

(611, 12)

In [19]:
perfect_cat_values = pd.concat([perfect_cat_values, num_values["Credit_History"]], axis = 1)
num_values = num_values.drop("Credit_History", axis = 1)

In [20]:
perfect_cat_values.head(2)

,Gender,Married,Self_Employed,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Credit_History
Loan_ID,,,,,,,,,,,,,
LP001002,1,0,0,True,False,False,False,True,False,False,False,True,1
LP001003,1,1,0,False,True,False,False,True,False,True,False,False,1


In [21]:
num_values.head(2)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term
Loan_ID,,,,
LP001002,5849,0.0,146.369492,360
LP001003,4583,1508.0,128.000000,360


In [22]:
num_values_cols = num_values.columns
list(num_values_cols)

['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']

In [23]:
std = StandardScaler()
num_values_arrays = std.fit_transform(num_values)

### `std.fit` calculates the standard deviation and mean of each columns in the dataset 
### `std.transform` applies the mean and standard deviation to each row in this fomula -> `(x-mean)/standard deviation`

In [24]:
num_values_arrays

array([[ 7.34964378e-02, -5.55745555e-01, -3.37663141e-16,
         2.75515640e-01],
       [-1.33509947e-01, -4.09552064e-02, -2.18238112e-01,
         2.75515640e-01],
       [-3.92349684e-01, -5.55745555e-01, -9.54826979e-01,
         2.75515640e-01],
       ...,
       [ 4.36983952e-01, -4.73816056e-01,  1.26682009e+00,
         2.75515640e-01],
       [ 3.57026510e-01, -5.55745555e-01,  4.82709358e-01,
         2.75515640e-01],
       [-1.33509947e-01, -5.55745555e-01, -1.58835784e-01,
         2.75515640e-01]])

In [25]:
standardized_num_values = pd.DataFrame(num_values_arrays, columns = list(num_values_cols), index = num_values.index)
standardized_num_values.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term
Loan_ID,,,,
LP001002,0.073496,-0.555746,-3.376631e-16,0.275516
LP001003,-0.133510,-0.040955,-2.182381e-01,0.275516
LP001005,-0.392350,-0.555746,-9.548270e-01,0.275516
LP001006,-0.460534,0.249212,-3.132818e-01,0.275516
LP001008,0.098187,-0.555746,-6.379206e-02,0.275516


In [26]:
restructured_x = pd.concat([standardized_num_values, perfect_cat_values], axis = 1)
restructured_x.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Gender,Married,Self_Employed,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Credit_History
Loan_ID,,,,,,,,,,,,,,,,,
LP001002,0.073496,-0.555746,-3.376631e-16,0.275516,1,0,0,True,False,False,False,True,False,False,False,True,1
LP001003,-0.133510,-0.040955,-2.182381e-01,0.275516,1,1,0,False,True,False,False,True,False,True,False,False,1
LP001005,-0.392350,-0.555746,-9.548270e-01,0.275516,1,1,1,True,False,False,False,True,False,False,False,True,1
LP001006,-0.460534,0.249212,-3.132818e-01,0.275516,1,1,0,True,False,False,False,False,True,False,False,True,1
LP001008,0.098187,-0.555746,-6.379206e-02,0.275516,1,0,0,True,False,False,False,True,False,False,False,True,1


In [27]:
full_df = pd.concat([restructured_x, y], axis =1)
full_df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Gender,Married,Self_Employed,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Credit_History,Loan_Status
Loan_ID,,,,,,,,,,,,,,,,,,
LP001002,0.073496,-0.555746,-3.376631e-16,0.275516,1,0,0,True,False,False,False,True,False,False,False,True,1,Y
LP001003,-0.133510,-0.040955,-2.182381e-01,0.275516,1,1,0,False,True,False,False,True,False,True,False,False,1,N
LP001005,-0.392350,-0.555746,-9.548270e-01,0.275516,1,1,1,True,False,False,False,True,False,False,False,True,1,Y
LP001006,-0.460534,0.249212,-3.132818e-01,0.275516,1,1,0,True,False,False,False,False,True,False,False,True,1,Y
LP001008,0.098187,-0.555746,-6.379206e-02,0.275516,1,0,0,True,False,False,False,True,False,False,False,True,1,Y


In [28]:
len(full_df.columns)

18

In [29]:
full_df["Loan_Status"] = full_df["Loan_Status"].apply(lambda x: 1 if x == "Y" else 0)

In [33]:
full_df.sample(10)

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Gender,Married,Self_Employed,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Credit_History,Loan_Status
Loan_ID,,,,,,,,,,,,,,,,,,
LP002053,-0.172916,-0.491226,-0.265760,0.275516,1,1,0,False,False,False,True,True,False,False,True,False,1,1
LP002288,-0.410500,-0.555746,-1.204317,-2.530152,1,1,0,False,False,True,False,False,True,False,False,True,0,0
LP001947,-0.493237,0.582392,0.304502,0.275516,1,1,0,True,False,False,False,True,False,False,True,False,1,1
LP001256,-0.269716,1.065776,0.352024,0.275516,1,0,0,True,False,False,False,True,False,False,False,True,1,0
LP002345,-0.715286,0.390882,-0.408326,0.275516,1,1,0,True,False,False,False,True,False,True,False,False,1,1
LP001120,-0.588564,-0.141660,-1.180556,0.275516,1,0,0,True,False,False,False,True,False,False,False,True,1,1
LP001109,-0.583986,-0.101720,-0.550891,0.275516,1,1,0,True,False,False,False,True,False,False,False,True,0,0
LP001197,-0.332504,0.195275,-0.135075,0.275516,1,1,0,True,False,False,False,True,False,True,False,False,1,0
LP001715,0.049624,-0.555746,-0.194477,0.275516,1,1,1,False,False,False,True,False,True,True,False,False,1,1


In [34]:
full_df.to_excel("../data/dataset_v2.xlsx" , index = None)